In [ ]:
from troposphere import Template, Ref, Tags, Output, Parameter, FindInMap, GetAtt, Join, Select, GetAZs, Base64
from troposphere.autoscaling import AutoScalingGroup, LaunchConfiguration, Tag
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancingv2 as elb

In [ ]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('autoscaling, application elb')
tags = Tags(Site='example.com');

# Params
---

In [ ]:
keyname_param = Parameter('keyName',
                          Default='main',
                          Description='existing key to use for ssh',
                          Type='String')
t.add_parameter(keyname_param);

# Maps
---

In [ ]:
t.add_mapping('RegionMap', { 'us-east-1': {'ami': 'ami-b73b63a0'}
                           , 'us-east-2': {'ami': 'ami-58277d3d'}
                           , 'us-west-1': {'ami': 'ami-23e8a343'}
                           , 'us-west-2': {'ami': 'ami-5ec1673e'}
                           });

# Resources
---

## Network

In [ ]:
vpc = ec2.VPC('myVpc', CidrBlock='10.0.0.0/16', Tags=tags)
resources.add(vpc)

In [ ]:
web_dmz1 = ec2.Subnet('webDmz1', 
                      CidrBlock='10.0.0.0/24', 
                      VpcId=Ref(vpc),
                      AvailabilityZone=Select('0', GetAZs(Ref('AWS::Region'))),
                      Tags=tags)

web_dmz2 = ec2.Subnet('webDmz2', 
                      CidrBlock='10.0.1.0/24', 
                      VpcId=Ref(vpc),
                      AvailabilityZone=Select('1', GetAZs(Ref('AWS::Region'))),
                      Tags=tags)
resources.add(web_dmz1)
resources.add(web_dmz2)

In [ ]:
igw = ec2.InternetGateway('igw', Tags=tags)
attach_igw = ec2.VPCGatewayAttachment('attachIgw', VpcId=Ref(vpc), InternetGatewayId=Ref(igw))
resources.add(igw)
resources.add(attach_igw)

In [ ]:
route_table = ec2.RouteTable('routeTable', VpcId=Ref(vpc), Tags=tags)
gw_route = ec2.Route('gwRoute',
                     DestinationCidrBlock='0.0.0.0/0',
                     GatewayId=Ref(igw),
                     RouteTableId=Ref(route_table))
associate_route_table1 = ec2.SubnetRouteTableAssociation('associateRouteTable1', 
                                                        RouteTableId=Ref(route_table),
                                                        SubnetId=Ref(web_dmz1))
associate_route_table2 = ec2.SubnetRouteTableAssociation('associateRouteTable2', 
                                                        RouteTableId=Ref(route_table),
                                                        SubnetId=Ref(web_dmz2))
resources.add(route_table)
resources.add(gw_route)
resources.add(associate_route_table1)
resources.add(associate_route_table2)

## Security Groups

In [ ]:
allowed_ports = [22, 80, 443]
rules = [ec2.SecurityGroupRule(CidrIp='0.0.0.0/0', FromPort=p, ToPort=p, IpProtocol='tcp') for p in allowed_ports]

web_dmz_sg = ec2.SecurityGroup('webDmzSg',
                               DependsOn='attachIgw',
                               GroupDescription='allow http/s traffic to web nodes',
                               VpcId=Ref(vpc),
                               SecurityGroupIngress=rules,
                               Tags=tags)
resources.add(web_dmz_sg)

## Autoscaling

In [ ]:
launch_config = LaunchConfiguration('launchConfig',
                                    DependsOn='attachIgw',
                                    SecurityGroups=[Ref(web_dmz_sg)],
                                    ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                                    InstanceType='t2.micro',
                                    KeyName=Ref(keyname_param))

asg_tags = [Tag('Site', 'example.com', True)]
web_asg = AutoScalingGroup('webAsg',
                           VPCZoneIdentifier=[Ref(web_dmz1), Ref(web_dmz2)],
                           LaunchConfigurationName=Ref(launch_config),
                           AvailabilityZones=[GetAtt(web_dmz1, 'AvailabilityZone'), GetAtt(web_dmz2, 'AvailabilityZone')],
                           DesiredCapacity=2,
                           MinSize=2,
                           MaxSize=4,
                           Tags=asg_tags)

resources.add(launch_config)
resources.add(web_asg)
launch_config.props

## Initial instances

In [ ]:
script = """\
#!/bin/bash
sudo yum update -y
sudo yum install httpd -y
wget https://raw.githubusercontent.com/epequeno/aws-cf/master/make_page.py
sudo python make_page.py
sudo service httpd start
"""

def make_userdata(text):
    return Base64(Join('', text.splitlines(True)))

user_data = make_userdata(script)

web_instance1 = ec2.Instance('web1',
                            ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                            InstanceType='t2.micro',
                            KeyName=Ref(keyname_param),
                            SecurityGroupIds=[GetAtt(web_dmz_sg, 'GroupId')],
                            SubnetId=Ref(web_dmz1),
                            UserData=user_data,
                            Tags=tags)

web_instance2 = ec2.Instance('web2',
                            ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                            InstanceType='t2.micro',
                            KeyName=Ref(keyname_param),
                            SecurityGroupIds=[GetAtt(web_dmz_sg, 'GroupId')],
                            SubnetId=Ref(web_dmz2),
                            UserData=user_data,
                            Tags=tags)

resources.add(web_instance1)
resources.add(web_instance2)

## Loadbalancing

In [ ]:
web_elb = elb.LoadBalancer('webElb',
                           Name='appElb',
                           Scheme='internet-facing',
                           Subnets=[Ref(web_dmz1), Ref(web_dmz2)],
                           SecurityGroups=[Ref(web_dmz_sg)],
                           Tags=tags)

web1_target_description = elb.TargetDescription('webTarget1',
                                                Id=Ref(web_instance1),
                                                Port=80)
web2_target_description = elb.TargetDescription('webTarget2',
                                                Id=Ref(web_instance2),
                                                Port=80)

web_target_group = elb.TargetGroup('webTargetGroup',
                                   Name='webTargetGroup',
                                   Port=80,
                                   Protocol='HTTP',
                                   HealthCheckIntervalSeconds=60,
                                   HealthCheckProtocol='HTTP',
                                   HealthCheckTimeoutSeconds=10,
                                   HealthyThresholdCount=4,
                                   UnhealthyThresholdCount=3,
                                   Targets=[web1_target_description, web2_target_description],
                                   VpcId=Ref(vpc),
                                   Matcher=elb.Matcher(HttpCode='200'),
                                   Tags=tags)

elb_listener = elb.Listener('elbListener',
                            Port='80',
                            Protocol='HTTP',
                            LoadBalancerArn=Ref(web_elb),
                            DefaultActions=[elb.Action(Type='forward',
                                                       TargetGroupArn=Ref(web_target_group))])

resources.add(web_elb)
resources.add(web_target_group)
resources.add(elb_listener)

# Outputs

# Build
---

In [ ]:
file_name = 'auto.json'

def build(filename):
    fname = 'outputs/' + filename
    for r in resources:
        t.add_resource(r)
    
    with open(fname, 'w') as fd:
        fd.write(t.to_json())
    
    !aws cloudformation validate-template --template-body file://{fname} > /dev/null && echo "build ok"

build(file_name)